<a href="https://colab.research.google.com/github/IPMSand/NLP-Module/blob/main/Sentiment_Analysis(2)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

# custom define

In [5]:
import re

In [1]:
# Lexciob:word -> sentiment score(positive > 0, negative <0)
lexicon = {
    'good': 1.0,
    'bad': -1.5,
    'terrible': -1.5,
    'love': 2.0,
    'hate': -2.0,
    'happy': 1.5,
    'sad': -1.5,

}


In [2]:
# Negation Words
negations =   {'not','no', 'never','none'}

In [6]:
def preprocess_text(text):
  """
  Preprocesses text by converting to lowercase and tokenizing into words.
  Uses re.findall as simple tokenizer to extract words.
  """
  text = text.lower()
  #
  words = re.findall(r'\b\w+\b', text)
  return words


def lexicon_sentiment(text):
  """
  Performs lexicon-basedd sentiment analysis with negation handling.
  Returns sentiment based on summed word scores.
  """
  words = preprocess_text(text)
  score = 0.0
  negate = False
  for word in words:
    if word in negations:
      negate = True # set flag to negate next sentiment word
      continue
    if word in lexicon:
      word_score = lexicon[word]
      if negate: # if negate is True
        word_score = -word_score # flip the polarity of the score
        negate = False
      score += word_score
  if score > 0:
    return 'positive'
  elif score < 0:
    return 'negative'
  else:
    return 'neutral'

In [12]:
# Example Usage
text = "I love this product, it's not bad at all!"
print(lexicon_sentiment(text)) # output : Positive

positive


In [13]:
# Example Usage 2
text = "I love this product, it's not bad at all!. I thought I will hate it really badly when I saw it's teribble state, Hate it!!"
print(lexicon_sentiment(text)) # Output

negative


# Pre Built

In [22]:
pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.5 MB/s eta 0:00:00


In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Initialize VADER
analyzer  = SentimentIntensityAnalyzer()

In [24]:
# Use VASER"S lexicon
def lexicon_sentiment_vader(text):
  scores = analyzer.polarity_scores(text)
  compound = scores['compound']
  if compound >= 0.05:
    return 'positive',compound
  elif compound <= -0.05:
    return 'negative', compound
  else:
    return 'neutral', compound


In [35]:
## Example Usage
texts = ["I love this product, i thought I will hate it.",
         "He is a cute bad boy",
         "Wow That's so sick!!",
         "wow That's so sick. Hate it",
         "ok, i guess, i will ses it later"
         ]
for text in texts:
  sentiment, score = lexicon_sentiment_vader(text)
  print(f"Text: {text}\nSentiment: {sentiment}, Score: {score}\n")

Text: I love this product, i thought I will hate it.
Sentiment: positive, Score: 0.128

Text: He is a cute bad boy
Sentiment: negative, Score: -0.128

Text: Wow That's so sick!!
Sentiment: positive, Score: 0.2001

Text: wow That's so sick. Hate it
Sentiment: negative, Score: -0.5819

Text: ok, i guess, i will ses it later
Sentiment: neutral, Score: 0.0

